In [13]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar
from scipy.integrate import simps


wavelength_range = [360,830]
def load_cmf(filename):
    data = pd.read_csv(filename, header=None, names=['wavelength', 'x', 'y', 'z'])
    cmf_wavelengths = data['wavelength'].values
    cmf_indices = np.where((cmf_wavelengths >= wavelength_range[0]) & (cmf_wavelengths <= wavelength_range[1]))[0]
    cmf_wavelengths = cmf_wavelengths[cmf_indices]
    x = np.interp(cmf_wavelengths, data['wavelength'].values, data['x'].values)
    y = np.interp(cmf_wavelengths, data['wavelength'].values, data['y'].values)
    z = np.interp(cmf_wavelengths, data['wavelength'].values, data['z'].values)
    cmf = {
        'wavelength': cmf_wavelengths,
        'x': x,
        'y': y,
        'z': z,
    }
    return cmf

def load_spd(filename, cmf):
    data = pd.read_csv(filename, sep='\t', skiprows=2, header=None, names=['wavelength', 'intensity'])
    spd_wavelengths = np.round(data['wavelength']).astype(int).values
    spd_indices = np.where((spd_wavelengths >= wavelength_range[0]) & (spd_wavelengths <= wavelength_range[1]))[0]
    spd_wavelengths = spd_wavelengths[spd_indices]
    spd_intensities = data['intensity'].values[spd_indices] 
    cmf_wavelengths = cmf['wavelength']
    x = np.interp(cmf_wavelengths, spd_wavelengths, spd_intensities)

    spd = {
        'wavelength': cmf_wavelengths,
        'intensity': x,
    }
    return spd

def spd_to_xyz(spd, wavelength, cmf):
    X = simps(spd * cmf['x'], wavelength)
    Y = simps(spd * cmf['y'], wavelength)
    Z = simps(spd * cmf['z'], wavelength)
    return np.array([X, Y, Z])

def planckian_locus(T, wavelength):
    h = 6.62607004e-34
    c = 3.0e8
    k = 1.38064852e-23
    nm_to_m = 1e-9
    T = T * np.ones_like(wavelength)
    B = (2 * h * c ** 2) / (wavelength * nm_to_m) ** 5 * (1 / (np.exp((h * c) / (wavelength * nm_to_m * k * T)) - 1))
    return B

def color_temperature(spd, wavelength, cmf):
    xyz = spd_to_xyz(spd, wavelength, cmf)
    x = xyz[0] / np.sum(xyz)
    y = xyz[1] / np.sum(xyz)

    def objective_function(T):
        B = planckian_locus(T, wavelength)
        xyz_blackbody = spd_to_xyz(B, wavelength, cmf)
        x_blackbody = xyz_blackbody[0] / np.sum(xyz_blackbody)
        y_blackbody = xyz_blackbody[1] / np.sum(xyz_blackbody)
        return (x - x_blackbody) ** 2 + (y - y_blackbody) ** 2

    result = minimize_scalar(objective_function, bounds=(1000, 100000), method='bounded')
    return result.x

# CIE 1931 2-deg Standard Observer Color Matching Functions (CMFs)
# Load CMF data
cmf_data = load_cmf('CIE_cc_1931_2deg.csv')

# Load SPD data
# spd_data = load_spd("14_white.txt",cmf_data)

# ct = color_temperature(spd_data['intensity'], spd_data['wavelength'] , cmf_data)
# print(f"Color temperature: {ct:.2f} K")

phone = []
color_temp = []

for i in range (11,15):
    dataset_filename = f"{i}_white.txt"
    # Load SPD data
    spd_data = load_spd(dataset_filename,cmf_data)
    ### calculate the color temperature
    ct = color_temperature(spd_data['intensity'], spd_data['wavelength'] , cmf_data)
    color_temp.append(ct)
    phone.append(f"iphone {i}")

table = pd.DataFrame(data={"Phone": phone, "Color temperature": color_temp})
print(table)


       Phone  Color temperature
0  iphone 11        5321.557679
1  iphone 12        5547.879565
2  iphone 13        5876.079574
3  iphone 14        5524.877969
